<hr style="border:2px solid #0176DE"> </hr>
<center><h1 style="color:#173F8A;"> Escuela de Verano en Metodos Iterativos</h1></center> 
<center><h1 style="color:#173F8A;"> EMI 2024 - CMM Chile</h1></center>
<hr style="border:2px solid #0176DE"> </hr>
<h3 style="color:#173F8A;text-align:right;"> Profesores: &nbsp;Nicolás Barnafi<br>Manuel A. Sanchez<br></h3>

<h3 style="color:#03122E;text-align:right;"> 
    Centro de Modelamiento Matematico <br> 
    Instituto de Ingenieria Matematica y Computacional - IMC UC<br>  
</h3>

<hr style="border:2px solid #03122E"> </hr>
<center><h1 style="color:#173F8A;"> Modulo 4: Precondicionadores</h1></center> 
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC:
Primaria: 
celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 
Secundaria
gris oscuro: #707070
-->

# Preconditioners

From $A=M - N$ we had
    $$ Ax = b \quad \Leftrightarrow Mx^k = Nx^{k-1} + b $$
This gives the increment equation for $\delta x^k = x^k - x^{k-1}$:
    $$ M\delta x^k = b - Ax^{k-1} $$
This is a Richardson iteration for the following system:
    $$ M^{-1}Ax = M^{-1}b $$
We call $P^{-1}:=M^{-1}$ the preconditioner. We seek that
    $$ \rho(P^{-1} A) << \rho(A) $$

# Preconditioned systems

$$ P^{-1}Ax = P^{-1}b $$
    
- The *ideal* preconditioner is $P=A$ as $\rho(P^{-1}A) = 1$. We settle with $P\approx A$.
- Preconditioning can be "left" or "right" (or "symmetrical", which is purely theoretical)
    $$ P^{-1}Ax = P^{-1}b \qquad \text{or} \qquad (AP^{-1})(Px) = b $$
  Difference: Left preconditioning modifies residual (important for error control)
- $P$ can be any linear operator. E.g.: some iterations of an iterative method
- Typical preconditioners: Jacobi, Gauss-Seidel, SOR, ILU, IC, AMG, DD

# Jacobi
$$P = \mathrm{diag} A = \begin{bmatrix}a_{11} & 0 &  \\ 0 & a_{22} & 0  \\ & & \ddots \end{bmatrix}$$
This means trivially that
$$ P^{-1} = (\mathrm{diag} A)^{-1} = \begin{bmatrix}1/a_{11} & 0 & & 0 \\ 0 & 1/a_{22} & 0 & \\ \end{bmatrix} $$

# Gauss-Seidel
$$ P = \mathrm{tril} A = \begin{bmatrix}a_{11} & 0 &  \\ a_{21} & a_{22} & 0  \\ & & \ddots \end{bmatrix}$$
This means that $P^{-1}$ is a forward substitution procedure.

# ILU

- TODO

# Block problems (multiphysics)

- TODO

In [34]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
import scipy.sparse as sp
from scipy.sparse.linalg import spilu
import matplotlib.pylab as plt
from ngsolve.la import EigenValues_Preconditioner

def getSystem(maxh, p):
    shape = Rectangle(1,1).Face()
    geo = OCCGeometry(shape, dim=2)
    mesh = Mesh(geo.GenerateMesh(maxh=maxh))
    V = H1(mesh, order=p)
    u, v = V.TnT()
    form = InnerProduct(Grad(u), Grad(v))*dx + u * v * dx
    a = BilinearForm(form).Assemble()
    f = LinearForm(v*dx).Assemble()
    gf = GridFunction(V)
    return a, f, gf, V

In [35]:
# We can define our own abstract preconditioners

class NoPrec(BaseMatrix):
    def __init__ (self, a):
        super(NoPrec, self).__init__()
        self.a = a
    def Mult (self, x, y):
        y[:] = x
    def Height (self):
        return self.a.mat.shape[0]
    def Width (self):
        return self.a.mat.shape[1]

class Jacobi(BaseMatrix):
    def __init__ (self, smoother, steps=1):
        super(Jacobi, self).__init__()
        self.smoother = smoother
        self.steps = steps
    def Mult (self, x, y):
        y[:] = 0.0
        y_work = x.Copy()
        for i in range(self.steps):
            self.smoother.Mult(y_work,y)
            y_work.Assign(y, 1)
    def Height (self):
        return self.smoother.height
    def Width (self):
        return self.smoother.height

class GaussSeidelL(BaseMatrix):
    def __init__ (self, smoother, steps=1):
        super(GaussSeidelL, self).__init__()
        self.smoother = smoother
        self.steps = steps
    def Mult (self, x, y):
        y[:] = 0.0
        y_work = x.Copy()
        for i in range(self.steps):
            self.smoother.Smooth(y, y_work)
            y_work.Assign(y, 1)
    def Height (self):
        return self.smoother.height
    def Width (self):
        return self.smoother.height

class GaussSeidelU(BaseMatrix):
    def __init__ (self, smoother, steps=1):
        super(GaussSeidelU, self).__init__()
        self.smoother = smoother
        self.steps = steps
    def Mult (self, x, y):
        y[:] = 0.0
        y_work = x.Copy()
        for i in range(self.steps):
            self.smoother.SmoothBack(y, y_work)
            y_work.Assign(y, 1)
    def Height (self):
        return self.smoother.height
    def Width (self):
        return self.smoother.height

class GaussSeidelSym(BaseMatrix):
    def __init__ (self, smoother, steps=1):
        super(GaussSeidelSym, self).__init__()
        self.smoother = smoother
        self.steps = steps
    def Mult (self, x, y):
        y[:] = 0.0
        y_work = x.Copy()
        for i in range(self.steps):
            self.smoother.Smooth(y, y_work)
            self.smoother.SmoothBack(y, y_work)
            y_work.Assign(y, 1)
    def Height (self):
        return self.smoother.height
    def Width (self):
        return self.smoother.height


class ILU(BaseMatrix):
    # Give a scipy.sparse mat
    def __init__ (self, a):
        super(ILU, self).__init__()
        rows,cols,vals = a.mat.COO()
        A = sp.csr_matrix((vals,(rows,cols))) # NGSolve mat to scipy.sparse
        self.A = A
        self.ilu = spilu(A)
    def Mult (self, x, y):
        x_vec = x.FV().NumPy()
        y_vec = self.ilu.solve(x_vec)
        y.FV()[:] = y_vec
    def Height (self):
        return self.A.shape[0]
    def Width (self):
        return self.A.shape[1]

def condest(a, prec):
    lams = EigenValues_Preconditioner(mat=a.mat, pre=prec)
    if lams:
        return max(lams)/min(lams)
    else:
        return None

In [58]:
def computeConditioning(maxh, p):
    a, f, gf, V = getSystem(maxh, p)
    preJpoint = a.mat.CreateSmoother(V.FreeDofs()) # smoothing operations such as Jacobi step or GS step
    steps = 1
    preNone = NoPrec(a)
    preJ = Jacobi(preJpoint, steps)
    preGSL = GaussSeidelL(preJpoint, steps)
    preGSU = GaussSeidelU(preJpoint, steps)
    preGSS = GaussSeidelSym(preJpoint, steps)
    preILU = ILU(a)
    
        
    
#solvers.CG(mat=a.mat, pre=preJ, rhs=f.vec, sol=gf.vec, tol=1e-6, maxsteps=50)
#solvers.GMRes(a.mat, f.vec, pre=preJ, x=gf.vec, tol=1e-6, maxsteps=50)

    #print(condest(a, preNone), condest(a, preJ), condest(a, preGSS), condest(a, preILU))
    names = ("None", "Jacobi","GSL", "GSU", "GSS", "ILU")
    precs = (preNone,preJ, preGSL, preGSU, preGSS, preILU)
    for name, prec in zip(names, precs):
        print("\n======== {}, estimated conditioning: {:1.2e}".format(name, condest(a,prec)))
        solvers.CG(mat=a.mat, pre=prec, rhs=f.vec, sol=gf.vec, printrates='\r', tol=1e-6, maxsteps=200)
        solvers.GMRes(a.mat, f.vec, pre=preJ, x=gf.vec, tol=1e-6, maxsteps=200, printrates='\r')

In [59]:
for h in [0.1, 0.05, 0.025, 0.0125]:
    print("\n==================== h =", h)
    computeConditioning(h, 1)


==================== h = 0.1

======== None, estimated conditioning: 7.19e+02
CG converged in 55 iterations to residual 5.735124459743455e-08
GMRes converged in 42 iterations to residual 6.367277693148646e-07

======== Jacobi, estimated conditioning: 6.65e+02
CG converged in 48 iterations to residual 4.7275524709179863e-08
GMRes converged in 42 iterations to residual 6.367277693148646e-07

======== GSL, estimated conditioning: -1.65e+00
CG NOT converged in 200 iterations to residual 0.18918985733619548
GMRes converged in 42 iterations to residual 6.367277693148646e-07

======== GSU, estimated conditioning: -1.66e+00
CG NOT converged in 200 iterations to residual 0.13814229290384897
GMRes converged in 42 iterations to residual 6.367277693148646e-07

======== GSS, estimated conditioning: 1.16e+02
CG converged in 22 iterations to residual 3.8899746841643835e-08
GMRes converged in 42 iterations to residual 6.367277693148646e-07

======== ILU, estimated conditioning: 1.00e+00
CG converged 

In [60]:
for p in [1,2,3,4]:
    print("\n==================== p =", p)
    computeConditioning(0.05, p)


==================== p = 1

======== None, estimated conditioning: 2.77e+03
CG converged in 99 iterations to residual 4.4580807243943486e-08
GMRes converged in 76 iterations to residual 9.191810124807026e-07

======== Jacobi, estimated conditioning: 2.75e+03
CG converged in 94 iterations to residual 1.8340825998201136e-08
GMRes converged in 76 iterations to residual 9.191810124807026e-07

======== GSL, estimated conditioning: -1.84e+00
CG NOT converged in 200 iterations to residual 0.33206635441339344
GMRes converged in 76 iterations to residual 9.191810124807026e-07

======== GSU, estimated conditioning: -1.93e+00
CG NOT converged in 200 iterations to residual 0.2536926179658144
GMRes converged in 76 iterations to residual 9.191810124807026e-07

======== GSS, estimated conditioning: 4.53e+02
CG converged in 41 iterations to residual 2.82763784664056e-08
GMRes converged in 76 iterations to residual 9.191810124807026e-07

======== ILU, estimated conditioning: 1.02e+00
CG converged in 4

# Intuiciones importantes

- El precondicionador debe aproximar la inversa
- La hipótesis de SPD de CG es FUNDAMENTAL
- No necesariamente GMRES funciona siempre mejor que CG
- Los smoothers pueden mejorar usando más iteraciones
- FEM genera problemas mal condicionados